# qRBM demo

In [45]:
%load_ext autoreload
%autoreload 2
from qRBM_final import qRBM

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Creating the underlying distribution
We flip an unbiased coin to generate a sequence of random bits, this will be the underlying hidden distribution of our data.

In [46]:
import pyquil.api as api
from random import *
import numpy as np

qvm = api.QVMConnection()

# Flip a coin.
np.random.seed(1234)
random_coin = np.random.choice([0,1], size=20, replace=True)
random_coin

array([1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0])

## Generating encoded sequences
Code below takes our sequence of random bits and encodes each 1 bit result into a 4 bit representation. This effectively creates artificial data with a 1 bit "needle" (hidden code subspace) in the "haystack" (4-bit data), which our RBM will have to decode. 

In [48]:
# Encode this coin flip in to an artifically high dimensional dataset.
artificial_data = []
for flip in random_coin:
    if flip == 1:
        artificial_data.append([1,1,-1,-1]) # Logical 1.
    else:
        artificial_data.append([-1,-1,1,1]) # Logical 0.

# We know have an artificially high dimensional dataset that still only
# has the 1 bit coin flip dictating the underlying distribution.
artificial_data = np.asarray(artificial_data)
artificial_data

array([[ 1,  1, -1, -1],
       [ 1,  1, -1, -1],
       [-1, -1,  1,  1],
       [ 1,  1, -1, -1],
       [-1, -1,  1,  1],
       [-1, -1,  1,  1],
       [-1, -1,  1,  1],
       [ 1,  1, -1, -1],
       [ 1,  1, -1, -1],
       [ 1,  1, -1, -1],
       [ 1,  1, -1, -1],
       [ 1,  1, -1, -1],
       [-1, -1,  1,  1],
       [-1, -1,  1,  1],
       [ 1,  1, -1, -1],
       [-1, -1,  1,  1],
       [-1, -1,  1,  1],
       [-1, -1,  1,  1],
       [-1, -1,  1,  1],
       [-1, -1,  1,  1]])

## Creating and training our qRBM

In [51]:
# We will now setup our RBM to try and understand the
# artifical data's underlying distribution.

# We will use "analytical" measurement to save time on simulation.
qr = qRBM(qvm, num_visible=4, num_hidden=1, n_quantum_measurements=None, verbose=True)

In [52]:
qr.train(artificial_data, n_epochs=10)

Beginning epoch 0
                     models will be ineffective
Optimization terminated successfully.
         Current function value: -0.035029
         Iterations: 2
         Function evaluations: 28
         Gradient evaluations: 7
Found following values for nus and gammas from QAOA:
[  3.90610868e-07]
[ 3.90882468]
--------------------------------------------------------------------------------
Found model expectation program...
POSITIVE PHASE
[[-0.04066144]
 [-0.04066144]
 [ 0.04066144]
 [ 0.04066144]]
NEGATIVE PHASE (QUANTUM)
[[ 0.58002566]
 [ 0.58002566]
 [ 0.58002566]
 [ 0.58002566]]
NEGATIVE PHASE (CLASSICAL)
[[ 0.2531396 ]
 [ 0.25749275]
 [ 0.2574254 ]
 [ 0.25107174]]
WEIGHTS
[[-0.01364317]
 [ 0.06251893]
 [ 0.06133963]
 [-0.04982246]]
--------------------------------------------------------------------------------

Beginning epoch 1
                     models will be ineffective
Optimization terminated successfully.
         Current function value: 0.000000
         Itera

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 4
         Function evaluations: 20
         Gradient evaluations: 5
Found following values for nus and gammas from QAOA:
[ 0.78539816]
[ 3.90882469]
--------------------------------------------------------------------------------
Found model expectation program...
POSITIVE PHASE
[[-0.09573401]
 [-0.09573401]
 [ 0.09573401]
 [ 0.09573401]]
NEGATIVE PHASE (QUANTUM)
[[  1.56598392e-17]
 [  7.95409465e-17]
 [  2.15678941e-17]
 [  4.38033560e-17]]
NEGATIVE PHASE (CLASSICAL)
[[ 0.23422994]
 [ 0.2374228 ]
 [ 0.24201012]
 [ 0.23734145]]
WEIGHTS
[[-0.12684758]
 [-0.05068548]
 [ 0.05853891]
 [-0.05262318]]
--------------------------------------------------------------------------------

Training Done!


## Comparing RBM-decoded data to originally encoded data
In the following code snippet we examine the hidden unit activation probabilities and how they correspond to the initial information from the coin flip. As we can see the probability of the hiddens corresponds very well with the flip.

In [53]:
# Now that the training is done (~5 mins when doing analytical expectation)
# we can transform our data to the hidden layer.
transformed = qr.transform(artificial_data)

comparison = np.stack((transformed[:, 0], random_coin))

# Compare our rbm probabilities with the coin flips.
print('RBM Pr. | Original Coin Value')
print('-'*20)
for i in range(len(transformed)):
    print('{: 0.3f}'.format(float(comparison[:,i][0])), ' |', '{: 0.3f}'.format(comparison[:,i][1]))


RBM Pr. | Original Coin Value
--------------------
 0.445  |  1.000
 0.445  |  1.000
 0.555  |  0.000
 0.445  |  1.000
 0.555  |  0.000
 0.555  |  0.000
 0.555  |  0.000
 0.445  |  1.000
 0.445  |  1.000
 0.445  |  1.000
 0.445  |  1.000
 0.445  |  1.000
 0.555  |  0.000
 0.555  |  0.000
 0.445  |  1.000
 0.555  |  0.000
 0.555  |  0.000
 0.555  |  0.000
 0.555  |  0.000
 0.555  |  0.000


# Thank you!
Check out 
https://github.com/MichaelBroughton/QABoM.
For a baseline version of this code that you can experiment with for yourself.